<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project2-TermDepositMarketing/2ndLayer_Regenerate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notes for Oct 22, <br>
SVC with SMOTEEN performing very well -> to conclude the 1st part, calculate the impact of the 1st model <br>
<br>
1. 1st layer)
  - One last step to go: Calculate how much time we will be able to save with the company and the % of subscribers we'll be loosing time, for all 40,000 customers to test model with the entire dataset. Here, I cannot use the call related time <br>
  - Consequence: SVC - RandomUnderSampler is the highest performance
<br>
2. 2nd layer) the precision might not be the best metrics, we can focus on f-1 score and find out a problem <br>
I had a problem here: If I train another test and split, the example might be different from that of the 1st layer. So maybe, instead of regenerating the test and train split, use the same test-train split set from the 1st layer but only apply few more columns.
<br>
3. 3rd layer)
 - do not use standardscaler, because I will loose interpretability (hard to understand the values)
 - try to print out average/ median for each clusters,
featrues I will be print out would be age, durations, jobs for these customers.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install optuna

In [ ]:
import os
import random

# Warnings
import warnings
warnings.filterwarnings("ignore")

# EDA
import pandas as pd
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings("ignore")

# Models
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
# from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
# from collections import Counter

# 2nd layer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE  # We use imbalanced-learn library.
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN

# 3rd layer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Validation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Read dataset
df = pd.read_csv('/content/drive/MyDrive/Apziva/TermDepositMarketing/term-deposit-marketing-2020.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1,no


## Models - 3 layers

1. 1st Layer: A layer to predict which of the potential customers we should make calls to (haven't make any calls yet,can't use any call-related features like duration, campaign, month of the call and etc).
2. 2nd Layer: A layer to predict which of the customers we should keep calling to. This means calls have already been made, I can use call-related data, to predict which of the customers we should keep making calls to. (precision for class 1)

They do not directly affect each other
3. 3rd Layer: Train both with the unsupervised model, to understand who those customers are and filtered the non-subscribers. The goal is to segment the subscribers.

In [ ]:
seed = random.randint(1000, 9999)
print(seed)

4505


In [ ]:
df_copy = df.copy()
df_copy

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1,no


### Train_test_split for Layer2

In [ ]:
for col in df.columns:
  print(col, df[col].unique())
  df_copy[col] = pd.factorize(df_copy[col])[0]
# df_copy

X2 = df_copy.drop(columns=['y'])   # The entire dataset is required so just drop Y.
# X2
y2 = df_copy['y']
y2

age [58 44 33 47 35 28 42 43 41 29 53 57 51 45 60 56 32 25 40 39 52 46 36 49
 59 37 50 54 55 48 24 38 31 30 27 34 23 26 61 22 21 20 66 62 83 75 67 70
 65 68 64 69 72 71 19 76 85 63 90 82 73 74 78 80 94 79 77 86 95 81]
job ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital ['married' 'single' 'divorced']
education ['tertiary' 'secondary' 'unknown' 'primary']
default ['no' 'yes']
balance [  2143     29      2 ...   7222   3402 102127]
housing ['yes' 'no']
loan ['no' 'yes']
contact ['unknown' 'cellular' 'telephone']
day [ 5  6  7  8  9 12 13 14 15 16 19 20 21 23 26 27 28 29 30  2  3  4 11 17
 18 24 25  1 10 22 31]
month ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr']
duration [ 261  151   76 ... 1880 1460 2219]
campaign [ 1  2  3  5  4  6  7  8  9 10 11 12 13 19 14 24 16 32 18 22 15 17 25 21
 43 51 63 41 26 28 55 50 38 23 20 29 31 37 30 46 27 58 33 35 34 36 39 44]


,y
0,0
1,0
2,0
3,0
4,0
...,...
39995,0
39996,1
39997,1
39998,0


In [ ]:
# Split the dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=seed)

In [ ]:
X2_train

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign
34033,0,5,0,1,0,4930,1,0,1,18,10,401,0
18358,29,8,0,1,0,2266,0,0,1,30,2,212,1
4564,33,3,0,1,0,559,0,0,0,11,0,127,1
17066,32,1,0,0,0,18,0,0,1,26,2,45,5
3220,6,9,0,0,0,75,0,0,0,8,0,439,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38241,35,1,0,1,0,300,1,0,1,8,0,395,0
7599,21,7,0,1,0,2673,0,1,0,18,0,247,0
30215,4,0,1,0,0,4381,1,0,1,0,8,312,0
11329,24,5,0,3,0,1501,1,0,0,24,1,158,1


### 2nd Layer: to predict which of the customers we should keep calling to <br>
Few tips to do: <br>
- Utilize the entire dataset with the complete set of all features. (Start from scratch) <br>
- Campaign related features: <br>
predict which of the customers we should keep calling, assuming initial round of calls have already made been -> campaign related features <br>

Similar to the previous steps: start with experimenting few different models.
Maybe try RandomUnderSampler, because it worked well and then experiment with few different models. <br>

This time, the goal is different, predicting which of the customer we should keep calling. Precision for class 1. (which of them precised correctly)

#### Data - RandomUnderSampler


|Data|Model||
|------|---|---|
|RUS (RandomUnderSampler)|RandomForestClassifier|
| |XGBClassifier|
| |SGDClassifier|
| |SVC|
| |LGBMClassifier|
| |DecisionTreeClassifier|


Techniques used in here:
1. RandomForestClassifier / XGBClassifier / SGDClassifier / SVC / LGBMClassifier/ DecisionTreeClassifier
2. SMOTE
3. SMOTE-Tomek
4. SMOTEEN

##### RandomForestClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)  # instead of using the entire dataset, apply o
print(y2_train_rus.value_counts())

# Initiate RandomForestClassifier
rfc_rus = RandomForestClassifier(random_state=seed)
rfc_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = rfc_rus.predict(X2_test)
print(f"RandomForestClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating RandomForestClassifier
rfc_sm = RandomForestClassifier(random_state=seed)
rfc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = rfc_sm.predict(X2_test)
print(f"RFC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating RandomForestClassifier
rfc_smt = RandomForestClassifier(random_state=seed)
rfc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = rfc_smt.predict(X2_test)
print(f"RFC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating RandomForestClassifier
rfc_smn = RandomForestClassifier(random_state=seed)
rfc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = rfc_smn.predict(X2_test)
print(f"RFC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2045
1    2045
Name: count, dtype: int64
RandomForestClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     11149
           1       0.25      0.79      0.38       851

    accuracy                           0.82     12000
   macro avg       0.62      0.81      0.64     12000
weighted avg       0.93      0.82      0.86     12000

y
0    25955
1    25955
Name: count, dtype: int64
RFC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94     11149
           1       0.34      0.47      0.39       851

    accuracy                           0.90     12000
   macro avg       0.65      0.70      0.67     12000
weighted avg       0.91      0.90      0.90     12000

y
0    25372
1    25372
Name: count, dtype: int64
RFC-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93    

##### XGBClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y1_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate XGBClassifier
xgb_rus = XGBClassifier(random_state=seed)
xgb_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = xgb_rus.predict(X2_test)
print(f"XGBClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating XGBClassifier
xgb_sm = XGBClassifier(random_state=seed)
xgb_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = xgb_sm.predict(X2_test)
print(f"XGB-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating XGBClassifier
xgb_smt = XGBClassifier(random_state=seed)
xgb_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = xgb_smt.predict(X2_test)
print(f"XGB-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating XGBClassifier
xgb_smn = XGBClassifier(random_state=seed)
xgb_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = xgb_smn.predict(X2_test)
print(f"XGB-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2038
1    2038
Name: count, dtype: int64
XGBClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     11142
           1       0.25      0.81      0.38       858

    accuracy                           0.81     12000
   macro avg       0.62      0.81      0.64     12000
weighted avg       0.93      0.81      0.85     12000

y
0    25962
1    25962
Name: count, dtype: int64
XGB-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94     11142
           1       0.33      0.48      0.39       858

    accuracy                           0.89     12000
   macro avg       0.65      0.70      0.67     12000
weighted avg       0.91      0.89      0.90     12000

y
0    25396
1    25396
Name: count, dtype: int64
XGB-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.96      0.92      0.94   

##### SGDClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate SGDClassifier
sgd_rus = SGDClassifier(random_state=seed)
sgd_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = sgd_rus.predict(X2_test)
print(f"SGDClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
sgd_sm = SGDClassifier(random_state=seed)
sgd_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = sgd_sm.predict(X2_test)
print(f"SGD-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SGDClassifier
sgd_smt = SGDClassifier(random_state=seed)
sgd_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = sgd_smt.predict(X2_test)
print(f"SGD-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SGDClassifier
sgd_smn = SGDClassifier(random_state=seed)
sgd_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = sgd_smn.predict(X2_test)
print(f"SGD-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2045
1    2045
Name: count, dtype: int64
SGDClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.94      0.98      0.96     11149
           1       0.31      0.11      0.17       851

    accuracy                           0.92     12000
   macro avg       0.62      0.55      0.56     12000
weighted avg       0.89      0.92      0.90     12000

y
0    25955
1    25955
Name: count, dtype: int64
SGD-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95     11149
           1       0.31      0.28      0.29       851

    accuracy                           0.90     12000
   macro avg       0.63      0.62      0.62     12000
weighted avg       0.90      0.90      0.90     12000

y
0    25372
1    25372
Name: count, dtype: int64
SGD-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.98      0.57      0.72   

##### SVC

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate SVC
svc_rus = SVC(random_state=seed)
svc_rus.fit(X2_train_rus, y2_train_rus)
y1_pred = svc_rus.predict(X2_test)
print(f"SVC Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
svc_sm = SVC(random_state=seed)
svc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = svc_sm.predict(X2_test)
print(f"SVC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SGDClassifier
svc_smt = SVC(random_state=seed)
svc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = svc_smt.predict(X2_test)
print(f"SVC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SVC
svc_smn = SVC(random_state=seed)
svc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = svc_smn.predict(X2_test)
print(f"SVC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2045
1    2045
Name: count, dtype: int64
SVC Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.94      0.98      0.96     11149
           1       0.31      0.11      0.17       851

    accuracy                           0.92     12000
   macro avg       0.62      0.55      0.56     12000
weighted avg       0.89      0.92      0.90     12000

y
0    25955
1    25955
Name: count, dtype: int64
SVC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.97      0.87      0.91     11149
           1       0.25      0.59      0.36       851

    accuracy                           0.85     12000
   macro avg       0.61      0.73      0.63     12000
weighted avg       0.91      0.85      0.87     12000

y
0    25372
1    25372
Name: count, dtype: int64
SVC-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.97      0.87      0.91     11149
  

##### LGBMClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate LGBMClassifier
lgbm_rus = LGBMClassifier(random_state=seed)
lgbm_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = lgbm_rus.predict(X2_test)
print(f"LGBMClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating LGBMClassifier
lgbm_sm = LGBMClassifier(random_state=seed)
lgbm_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = lgbm_sm.predict(X2_test)
print(f"LGBM-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating LGBMClassifier
lgbm_smt = LGBMClassifier(random_state=seed)
lgbm_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = svc_smt.predict(X2_test)
print(f"LGBM-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating LGBMClassifier
lgbm_smn = SVC(random_state=seed)
lgbm_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = lgbm_smn.predict(X2_test)
print(f"LGBM-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2045
1    2045
Name: count, dtype: int64
[LightGBM] [Info] Number of positive: 2045, number of negative: 2045
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000970 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 662
[LightGBM] [Info] Number of data points in the train set: 4090, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LGBMClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.83      0.90     11149
           1       0.26      0.80      0.40       851

    accuracy                           0.83     12000
   macro avg       0.62      0.81      0.65     12000
weighted avg       0.93      0.83      0.86     12000

y
0    25955
1    25955
Name: count, dtype: int64
[LightGBM] [Info] Number o

##### DecisionTreeClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate DecisionTreeClassifier
dtc_rus = DecisionTreeClassifier(random_state=seed)
dtc_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = dtc_rus.predict(X2_test)
print(f"DecisionTreeClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating DTClassifier
dtc_sm = DecisionTreeClassifier(random_state=seed)
dtc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = dtc_sm.predict(X2_test)
print(f"DTC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating DTClassifier
dtc_smt = DecisionTreeClassifier(random_state=seed)
dtc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = dtc_smt.predict(X2_test)
print(f"DTC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating DTClassifier
dtc_smn = SVC(random_state=seed)
dtc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = dtc_smn.predict(X2_test)
print(f"DTC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2045
1    2045
Name: count, dtype: int64
DecisionTreeClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.97      0.73      0.83     11149
           1       0.17      0.73      0.28       851

    accuracy                           0.73     12000
   macro avg       0.57      0.73      0.55     12000
weighted avg       0.92      0.73      0.79     12000

y
0    25955
1    25955
Name: count, dtype: int64
DTC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.95      0.89      0.92     11149
           1       0.23      0.43      0.30       851

    accuracy                           0.86     12000
   macro avg       0.59      0.66      0.61     12000
weighted avg       0.90      0.86      0.88     12000

y
0    25372
1    25372
Name: count, dtype: int64
DTC-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.95      0.89    

#### Data - RandomOverSampler


|Data|Model||
|------|---|---|
|RUS (RandomUnderSampler)|RandomForestClassifier|
| |XGBClassifier|
| |SGDClassifier|
| |SVC|
| |LGBMClassifier|
| |DecisionTreeClassifier|


Techniques used in here:
1. RandomForestClassifier / XGBClassifier / SGDClassifier / SVC / LGBMClassifier/ DecisionTreeClassifier
2. SMOTE
3. SMOTE-Tomek
4. SMOTEEN

##### RandomForestClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = rus.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate RandomForestClassifier
rfc_ros = RandomForestClassifier(random_state=seed)
rfc_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = rfc_ros.predict(X2_test)
print(f"RandomForestClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating RandomForestClassifier
rfc_sm = RandomForestClassifier(random_state=seed)
rfc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = rfc_sm.predict(X2_test)
print(f"RFC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating RandomForestClassifier
rfc_smt = RandomForestClassifier(random_state=seed)
rfc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = rfc_smt.predict(X2_test)
print(f"RFC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating RandomForestClassifier
rfc_smn = RandomForestClassifier(random_state=seed)
rfc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = rfc_smn.predict(X2_test)
print(f"RFC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    2045
1    2045
Name: count, dtype: int64
RandomForestClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     11149
           1       0.25      0.79      0.38       851

    accuracy                           0.82     12000
   macro avg       0.62      0.81      0.64     12000
weighted avg       0.93      0.82      0.86     12000

y
0    25955
1    25955
Name: count, dtype: int64
RFC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94     11149
           1       0.34      0.47      0.39       851

    accuracy                           0.90     12000
   macro avg       0.65      0.70      0.67     12000
weighted avg       0.91      0.90      0.90     12000

y
0    25372
1    25372
Name: count, dtype: int64
RFC-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93    

##### XGBClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate XGBClassifier
xgb_ros = XGBClassifier(random_state=seed)
xgb_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = xgb_ros.predict(X2_test)
print(f"XGBClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y1_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating XGBClassifier
xgb_sm = XGBClassifier(random_state=seed)
xgb_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = xgb_sm.predict(X2_test)
print(f"XGB-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating XGBClassifier
xgb_smt = XGBClassifier(random_state=seed)
xgb_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = xgb_smt.predict(X2_test)
print(f"RFC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating RandomForestClassifier
xgb_smn = XGBClassifier(random_state=seed)
xgb_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = xgb_smn.predict(X2_test)
print(f"RFC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    25955
1    25955
Name: count, dtype: int64
XGBClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.97      0.91      0.94     11149
           1       0.34      0.63      0.44       851

    accuracy                           0.89     12000
   macro avg       0.66      0.77      0.69     12000
weighted avg       0.93      0.89      0.90     12000

y
0    25955
1    25955
Name: count, dtype: int64
XGB-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     11149
           1       0.32      0.47      0.38       851

    accuracy                           0.89     12000
   macro avg       0.64      0.70      0.66     12000
weighted avg       0.91      0.89      0.90     12000

y
0    25372
1    25372
Name: count, dtype: int64
RFC-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.96      0.92      0.94 

##### SGDClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate SGDClassifier
sgd_ros = SGDClassifier(random_state=seed)
sgd_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = sgd_ros.predict(X2_test)
print(f"SGDClassifier Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
sgd_sm = SGDClassifier(random_state=seed)
sgd_sm.fit(X2_train_sm, y2_train_sm)
y1_pred_sm = sgd_sm.predict(X2_test)
print(f"SGD-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SGDClassifier
sgd_smt = SGDClassifier(random_state=seed)
sgd_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = sgd_smt.predict(X2_test)
print(f"SGD-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SGDClassifier
sgd_smn = SGDClassifier(random_state=seed)
sgd_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = sgd_smn.predict(X2_test)
print(f"SGD-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    25955
1    25955
Name: count, dtype: int64
SGDClassifier Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.95      0.96      0.95     11149
           1       0.33      0.27      0.30       851

    accuracy                           0.91     12000
   macro avg       0.64      0.62      0.63     12000
weighted avg       0.90      0.91      0.91     12000

y
0    25955
1    25955
Name: count, dtype: int64
SGD-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     11149
           1       0.32      0.47      0.38       851

    accuracy                           0.89     12000
   macro avg       0.64      0.70      0.66     12000
weighted avg       0.91      0.89      0.90     12000

y
0    25372
1    25372
Name: count, dtype: int64
SGD-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.98      0.57      0.72  

##### SVC

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate SVC
svc_ros = SVC(random_state=seed)
svc_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = svc_ros.predict(X2_test)
print(f"SVC Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
svc_sm = SVC(random_state=seed)
svc_sm.fit(X2_train_sm, y2_train_sm)
y1_pred_sm = svc_sm.predict(X2_test)
print(f"SVC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SVC
svc_smt = SVC(random_state=seed)
svc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = svc_smt.predict(X2_test)
print(f"SVC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SVC
svc_smn = SVC(random_state=seed)
svc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = svc_smn.predict(X2_test)
print(f"SVC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))


# Somthing's wrong here:
# For the 1st layer, I had a higher perforamcne with fewer features, but here, I train with many columns but!! having a lower performancce.
# I should detect why - probably small mistakes!

y
0    25955
1    25955
Name: count, dtype: int64
SVC Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.97      0.87      0.91     11149
           1       0.25      0.59      0.36       851

    accuracy                           0.85     12000
   macro avg       0.61      0.73      0.64     12000
weighted avg       0.91      0.85      0.87     12000

y
0    25955
1    25955
Name: count, dtype: int64
SVC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.92      0.94     11149
           1       0.32      0.47      0.38       851

    accuracy                           0.89     12000
   macro avg       0.64      0.70      0.66     12000
weighted avg       0.91      0.89      0.90     12000

y
0    25372
1    25372
Name: count, dtype: int64
SVC-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.97      0.87      0.91     11149
 

##### LGBMClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate LGBMClassifier
lgbm_ros = LGBMClassifier(random_state=seed)
lgbm_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = lgbm_ros.predict(X2_test)
print(f"LGBMClassifier Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating LGBMClassifier
lgbm_sm = LGBMClassifier(random_state=seed)
lgbm_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = lgbm_sm.predict(X2_test)
print(f"LGBM-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating LGBM
lgbm_smt = SVC(random_state=seed)
lgbm_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = lgbm_smt.predict(X2_test)
print(f"LGBM-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating LGBM
lgbm_smn = LGBMClassifier(random_state=seed)
lgbm_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = svc_smn.predict(X2_test)
print(f"LGBM-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    25955
1    25955
Name: count, dtype: int64
[LightGBM] [Info] Number of positive: 25955, number of negative: 25955
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003989 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 684
[LightGBM] [Info] Number of data points in the train set: 51910, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LGBMClassifier Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.98      0.87      0.92     11149
           1       0.31      0.77      0.44       851

    accuracy                           0.86     12000
   macro avg       0.65      0.82      0.68     12000
weighted avg       0.93      0.86      0.89     12000

y
0    25955
1    25955
Name: count, dtype: int64
[LightGBM] [Info] Numb

##### DecisionTreeClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate DecisionTreeClassifier
dtc_ros = DecisionTreeClassifier(random_state=seed)
dtc_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = dtc_ros.predict(X2_test)
print(f"DecisionTreeClassifier Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating DecisionTreeClassifier
dtc_sm = DecisionTreeClassifier(random_state=seed)
dtc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = dtc_sm.predict(X2_test)
print(f"DTC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating DecisionTreeClassifier
dtc_smt = DecisionTreeClassifier(random_state=seed)
dtc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = dtc_smt.predict(X2_test)
print(f"LGBM-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating DTC
dtc_smn = DecisionTreeClassifier(random_state=seed)
dtc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = dtc_smn.predict(X2_test)
print(f"DTC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))

y
0    25955
1    25955
Name: count, dtype: int64
DecisionTreeClassifier Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95     11149
           1       0.31      0.30      0.30       851

    accuracy                           0.90     12000
   macro avg       0.63      0.62      0.63     12000
weighted avg       0.90      0.90      0.90     12000

y
0    25955
1    25955
Name: count, dtype: int64
DTC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.95      0.89      0.92     11149
           1       0.23      0.43      0.30       851

    accuracy                           0.86     12000
   macro avg       0.59      0.66      0.61     12000
weighted avg       0.90      0.86      0.88     12000

y
0    25372
1    25372
Name: count, dtype: int64
LGBM-SMOTE TOMEK Applied: 
               precision    recall  f1-score   support

           0       0.95      0.89  

### 3rd Layer: Training, help the company understand who those customers are
- filter out non subscribers and only work with subscribers
- unsupversied training K-means and hierarchical clustering to identify customer segments

#### K-Means and PCA

In [ ]:
# Step1: Filter the data to include only subscribers.
subscribers_data = df_copy[df_copy['y']==1]  # Assuming 'y' is the column indicating subscribers
# subscribers_data  # 2896


# Step 2: Select relevant features for clustering
# Exclude 'y' and other call-related features like 'duration', 'campaign', etc.
clustering_features = subscribers_data.drop(columns=['y', 'duration', 'campaign', 'day', 'month'])
# clustering_features  # 2896


# Step 3: Scale the data
# StandardScaler = StandardScaler()
scaled_features = StandardScaler.fit_transform(clustering_features)
# scaled_features

# Step 4: Apply KMeans Clustering
# Determine optimal number of clusters using the elbow method
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=seed)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Choose the number of clusters (e.g., 3) based on the elbow plot
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, random_state=seed)
subscribers_data['Cluster'] = kmeans.fit_predict(scaled_features)


# Step 5: Analyze and interpret the clusters
# For example, checking the distribution of each cluster:
print(subscribers_data['Cluster'].value_counts())

# Or calculating the mean values of each feature within clusters to understand characteristics:
cluster_summary = subscribers_data.groupby('Cluster').mean()
# For Ordinal Feature:
# Job/Education etc.. Feature: try to calculate the most common and include that separately unless it makse hard for me to interpret the data
print(cluster_summary)

TypeError: TransformerMixin.fit_transform() missing 1 required positional argument: 'X'

In [ ]:
# If needed, visualize the clusters using PCA (Principal Component Analysis) for better understanding:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_components = pca.fit_transform(scaled_features)

plt.figure(figsize=(10, 6))
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=subscribers_data['Cluster'], cmap='viridis')
plt.title('Customer Segmentation using KMeans')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster')
plt.show()

#### K-Means

In [ ]:
# Step 1: Filter the data to include only subscribers
subscribers_data = df_copy[df_copy['y'] == 1]  # Assuming 'y' is the column indicating subscription status

# Step 2: Select relevant features for clustering
# Exclude 'y' and other call-related features like 'duration', 'campaign', 'day', 'month'
clustering_features = subscribers_data.drop(columns=['y', 'duration', 'campaign', 'day', 'month'])

# Step 3: Scale the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(clustering_features)

# Step 4: Apply KMeans Clustering
# Determine the optimal number of clusters using the elbow method (optional)
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Choose the number of clusters (e.g., 3) based on the elbow plot
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
subscribers_data['Cluster'] = kmeans.fit_predict(scaled_features)

# Step 5: Analyze and interpret the clusters
print(subscribers_data['Cluster'].value_counts())

# Calculate the mean values of each feature within clusters to understand characteristics
cluster_summary = subscribers_data.groupby('Cluster').mean()
print(cluster_summary)

#### Hierarhical Clustering

In [ ]:
# Step 1: Filter the data to include only subscribers
subscribers_data = df_copy[df_copy['y'] == 1]  # Assuming 'y' is the column indicating subscription status

# Step 2: Select relevant features for clustering
# Exclude 'y' and other call-related features like 'duration', 'campaign', 'day', 'month'
clustering_features = subscribers_data.drop(columns=['y', 'duration', 'campaign', 'day', 'month'])

# Step 3: Scale the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(clustering_features)

# Step 4: Apply Hierarchical Clustering
# Create the linkage matrix using 'ward' method
linked = linkage(scaled_features, method='ward')

# Plot the dendrogram to determine the optimal number of clusters
plt.figure(figsize=(10, 7))
dendrogram(linked)
plt.title('Dendrogram')
plt.xlabel('Samples')
plt.ylabel('Euclidean distances')
plt.show()

# Cut the dendrogram at a point to define clusters, e.g., 3 clusters
optimal_clusters = 3
hierarchical = AgglomerativeClustering(n_clusters=optimal_clusters, linkage='ward')
subscribers_data['Cluster'] = hierarchical.fit_predict(scaled_features)

# Step 5: Analyze and interpret the clusters
print(subscribers_data['Cluster'].value_counts())
# But I don't know what each values means

# Calculate the mean values of each feature within clusters to understand characteristics
cluster_summary = subscribers_data.groupby('Cluster').mean()
print(cluster_summary)

In [ ]:
# Step 1: Filter out non-subscribers (only keep customers who have purchased the product)
subscribers_df = df_copy[df_copy['y'] == 1]

# Step 2: Select features for clustering (you can adjust these based on relevance)
features = ['age', 'balance', 'duration', 'campaign']  # Example features
X = subscribers_df[features]

# Step 3: Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

# Step 4: Determine the optimal number of clusters using the elbow method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Step 5: Apply K-means with the chosen number of clusters (e.g., k=3 based on elbow plot)
kmeans = KMeans(n_clusters=3, random_state=42)
subscribers_df['cluster'] = kmeans.fit_predict(scaled_features)

# Step 6: Analyze the clusters
# Get the mean values of each feature for each cluster to understand their characteristics
cluster_summary = subscribers_df.groupby('cluster')[features].mean()
print(cluster_summary)

# Step 7: Visualize the segments
# This can be done using bar plots, scatter plots, etc.
# For example, to see how 'balance' and 'duration' differ across clusters:
sns.boxplot(x='cluster', y='balance', data=subscribers_df)
plt.title('Balance Distribution Across Clusters')
plt.show()

sns.boxplot(x='cluster', y='duration', data=subscribers_df)
plt.title('Duration Distribution Across Clusters')
plt.show()

# 1. Cluster 0:
# Median Balance: The median balance is relatively low compared to the other clusters.
# Balance Range: The interquartile range (IQR) is small, indicating less variability in balances within this cluster.
# Potential Insight: Customers in this cluster have lower balances, which could mean they might be less likely to invest large amounts or prefer smaller deposits. They might need more encouragement to invest.

# 2. Cluster 1:
# Median Balance: The median balance is the highest among all clusters.
# Balance Range: The IQR is wider, meaning there's a larger spread of balances among customers in this cluster. The balance is generally high.
# Potential Insight: This cluster represents customers with higher balances, which makes them potentially valuable targets for investment products. They might have more disposable income and could be more willing to invest in higher-value products.

# 3. Cluster 2:
# Median Balance: This cluster has a median balance between Cluster 0 and Cluster 1.
# Balance Range: There is a moderate spread in the balances, with some outliers on the higher end.
# Potential Insight: Customers in this cluster have moderate balances but could have a few individuals who are potentially high-value investors. They might be interested in products with medium-term returns or might need targeted offers to convert.